In [1]:
import pint.toa
import pint.models
import pint.fitter
import pint.residuals
import pint.utils
import pint.models.model_builder as mb
import pint.random_models
from pint.phase import Phase
from copy import deepcopy
from collections import OrderedDict
from astropy import log
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import operator
import time
from pathlib import Path
#import APT_argparse
import socket

In [43]:
f = pint.fitter.WLSFitter(t, m)

f.get_designmatrix()

array([   0.40726644, -215.77663308])

In [2]:
from APT_binary import JUMP_adder_begginning, starting_points
os.chdir("/Users/jackson/Desktop/Pint_Personal/APT/binary3_1")

parfile = Path("fake_1.par")
timfile = Path("fake_1.tim")
m, t = mb.get_model_and_toas(parfile, timfile)
t.table["clusters"] = t.get_clusters()
mask = starting_points(toas = t)[0]
t, m = JUMP_adder_begginning(mask[0], t, m, "thisisatest.tim", "thisisatest.par")
t.table['flags']

2022-06-23 15:20:54.725 | DEBUG    | pint.toa:__init__:1304 - No pulse number flags found in the TOAs
2022-06-23 15:20:54.726 | DEBUG    | pint.toa:apply_clock_corrections:2085 - Applying clock corrections (include_gps = True, include_bipm = True)
2022-06-23 15:20:54.740 | INFO     | pint.observatory.global_clock_corrections:get_file:121 - File index.txt to be downloaded due to download policy if_expired: https://raw.githubusercontent.com/ipta/pulsar-clock-corrections/main/index.txt
2022-06-23 15:20:54.882 | DEBUG    | pint.observatory.clock_file:__init__:566 - Loading tempo observatory (None) clock correction file /Users/jackson/.astropy/cache/download/url/599e3ebbfc317e090244ee1ef4c79374/contents
2022-06-23 15:20:54.933 | INFO     | pint.observatory.topo_obs:find_clock_file:464 - Using global clock file for time_gbt.dat
2022-06-23 15:20:54.936 | INFO     | pint.observatory.topo_obs:clock_corrections:258 - Applying observatory clock corrections.
2022-06-23 15:20:54.941 | INFO     | pi

"{'format': 'Tempo2', 'name': 'fake', 'clkcorr': '2.7611500005802326e-05'}"
"{'format': 'Tempo2', 'name': 'fake', 'clkcorr': '2.7611464983094113e-05'}"
"{'format': 'Tempo2', 'name': 'fake', 'clkcorr': '2.761142999056391e-05'}"
"{'format': 'Tempo2', 'name': 'fake', 'clkcorr': '2.761139496783465e-05'}"
"{'format': 'Tempo2', 'name': 'fake', 'clkcorr': '2.7611359948829142e-05'}"
"{'format': 'Tempo2', 'name': 'fake', 'clkcorr': '2.761132492611692e-05'}"
"{'format': 'Tempo2', 'name': 'fake', 'clkcorr': '2.7611289933511566e-05'}"
"{'format': 'Tempo2', 'name': 'fake', 'clkcorr': '2.7608480209963185e-05', 'jump_tim': '1'}"
"{'format': 'Tempo2', 'name': 'fake', 'clkcorr': '2.760851408196982e-05', 'jump_tim': '1'}"
"{'format': 'Tempo2', 'name': 'fake', 'clkcorr': '2.7608547957674082e-05', 'jump_tim': '1'}"
"{'format': 'Tempo2', 'name': 'fake', 'clkcorr': '2.7608581829680227e-05', 'jump_tim': '1'}"


In [3]:
m.as_parfile() # adding jumps natively gives errors so I will add them manually
with open("thisisatest1.par", "w") as parfile:
    parfile.write(m.as_parfile())
    for i in range(1, 31):
        parfile.write(f"JUMP\t\t-jump_tim {i}\t0 1 0\n")
    

In [2]:
from APT_binary import JUMP_adder_begginning, starting_points
os.chdir("/Users/jackson/Desktop/Pint_Personal/APT/binary3_1")

parfile = Path("fake_1.par")
timfile = Path("fake_1.tim")
m, t = mb.get_model_and_toas(parfile, timfile)
t.table["clusters"] = t.get_clusters()
mask = starting_points(toas = t)[0]
t, m = JUMP_adder_begginning(mask[0], t, m, "thisisatest.tim", "thisisatest.par")

2022-06-23 14:55:50.486 | DEBUG    | pint.toa:__init__:1304 - No pulse number flags found in the TOAs
2022-06-23 14:55:50.488 | DEBUG    | pint.toa:apply_clock_corrections:2085 - Applying clock corrections (include_gps = True, include_bipm = True)
2022-06-23 14:55:50.507 | DEBUG    | pint.observatory.clock_file:__init__:566 - Loading tempo observatory (None) clock correction file /Users/jackson/.astropy/cache/download/url/599e3ebbfc317e090244ee1ef4c79374/contents
2022-06-23 14:55:50.548 | INFO     | pint.observatory.topo_obs:find_clock_file:464 - Using global clock file for time_gbt.dat
2022-06-23 14:55:50.549 | INFO     | pint.observatory.topo_obs:clock_corrections:258 - Applying observatory clock corrections.
2022-06-23 14:55:50.551 | INFO     | pint.observatory.topo_obs:clock_corrections:263 - Applying GPS to UTC clock correction (~few nanoseconds)
2022-06-23 14:55:50.551 | INFO     | pint.observatory.topo_obs:_load_gps_clock:190 - Loading global GPS clock file for gbt
2022-06-23 14

In [3]:
m.components

{'AbsPhase': AbsPhase(
     MJDParameter(   TZRMJD              56000.0000000000000000 (d) frozen=True),
     strParameter(   TZRSITE             GBT               frozen=True),
     floatParameter( TZRFRQ              1400.0            (MHz) frozen=True)),
 'Spindown': Spindown(
     floatParameter( F0                  2.4553950186475038 (Hz) +/- 1e-06 Hz frozen=False),
     MJDParameter(   PEPOCH              56000.0000000000000000 (d) frozen=True),
     floatParameter( F1                  0.0               (Hz / s) +/- 0.0 Hz / s frozen=True)),
 'PhaseJump': PhaseJump(
     maskParameter(JUMP1 -jump_tim 1 0.0 +/- 0.0 s (s)),
     maskParameter(JUMP2 -jump_tim 2 0.0 +/- 0.0 s (s)),
     maskParameter(JUMP3 -jump_tim 3 0.0 +/- 0.0 s (s)),
     maskParameter(JUMP4 -jump_tim 4 0.0 +/- 0.0 s (s)),
     maskParameter(JUMP5 -jump_tim 5 0.0 +/- 0.0 s (s)),
     maskParameter(JUMP6 -jump_tim 6 0.0 +/- 0.0 s (s)),
     maskParameter(JUMP7 -jump_tim 7 0.0 +/- 0.0 s (s)),
     maskParameter(JUM

In [36]:
# Now to test out the design matrix

m1 = deepcopy(m)
t1 = deepcopy(t)

f1 = pint.fitter.WLSFitter(t1, m1)
# f = pint.fitter.WLSFitter(t, m)
# f.get_designmatrix()
m1.components

AttributeError: Attribute JUMP not found in TimingModel or any Component

In [ ]:
def JUMP_adder_begginning_cluster(
    mask: np.ndarray, toas, model, output_timfile, output_parfile
):
    """
    Adds JUMPs to a timfile as the begginning of analysis.

    mask : a mask to select which toas will not be jumped
    toas : TOA object
    output_timfile : name for the tim file to be written
    output_parfile : name for par file to be written
    """
    t = deepcopy(toas)
    flag_name = "jump_tim"

    former_cluster = t.table[mask]["clusters"][0]
    j = 0
    for i, table in enumerate(t.table[~mask]):
        # if table["clusters"] != former_cluster:
        #     former_cluster = table["clusters"]
        #     j += 1
        table["flags"][flag_name] = str(table["clusters"])
    t.write_TOA_file(output_timfile)

    # model.jump_flags_to_params(t) doesn't currently work (need flag name to be "tim_jump" and even then it still won't work),
    # so the following is a workaround. This is likely related to issue 1294.
    ### (workaround surrounded in ###)
    with open(output_parfile, "w") as parfile:
        parfile.write(model.as_parfile())
        for i in range(1, j + 1):
            parfile.write(f"JUMP\t\t-{flag_name} {i}\t0 1 0\n")
    model = mb.get_model(output_parfile)
    ###

    return t, model

os.chdir("/Users/jackson/Desktop/Pint_Personal/APT/binary3_1")

parfile = Path("fake_1.par")
timfile = Path("fake_1.tim")
m, t = mb.get_model_and_toas(parfile, timfile)
t.table["clusters"] = t.get_clusters()
mask = starting_points(toas = t)[0]
t, m = JUMP_adder_begginning_cluster(mask[0], t, m, "thisisatest.tim", "thisisatest.par")

In [10]:
masks_list, start_clusters_list = starting_points(t)
for mask_number, mask in enumerate(masks_list):
    print(mask_number)

starting_points(t)[1]

0
1
2
3
4


[0, 9, 10, 14, 19]

In [60]:
([12,3], [4,3,4])[0]

[12, 3]